# Couche silver
transformer les données puis les stocker dans hive

In [42]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, date_format, col
# Step 1: Create Spark session with Hive support
spark = SparkSession.builder \
    .appName("SilverLayer") \
    .enableHiveSupport() \
    .getOrCreate()

Effectuer des modifications sur le jeu de données


In [43]:
music_info_df = spark.read.parquet("../Bronze/data/csv", header=True, inferSchema=True) #Load the csv file containing the data 

#
music_info_df.describe().show()



clean_df = music_info_df.withColumn("release_year", date_format("release_date", "yyyy"))\
    .drop("year")\
    .drop("release_date")\
    .drop("mode")\
    .drop("explicit")\
    .dropna()\
    .dropDuplicates()

df_1921 = clean_df.filter(col("year") == 1921)

clean_df.describe().show()


+-------+-------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------------------+------------------+----------------------+------------------+--------------------+--------------------+--------------------+
|summary|            valence|              year|      acousticness|             artists|        danceability|         duration_ms|              energy|            explicit|                  id|    instrumentalness|               key|            liveness|           loudness|              mode|                  name|        popularity|        release_date|         speechiness|               tempo|
+-------+-------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------

Save data to a hive database

In [44]:
spark.sql("CREATE DATABASE IF NOT EXISTS spotifyInfo")

clean_df.write \
    .mode("overwrite") \
    .option("path", "data/musicInfo")\
    .format("parquet") \
    .saveAsTable("spotifyInfo.musicInfo")

spark.sql("SELECT * FROM spotifyInfo.musicInfo").describe().show()
spark.sql("SELECT * FROM spotifyInfo.musicInfo").show()


+-------+-------------------+-------------------+--------------------+-------------------+------------------+-------------------+--------------------+-------------------+-----------------+-------------------+--------------------+----------------------+-----------------+-------------------+------------------+------------------+
|summary|            valence|       acousticness|             artists|       danceability|       duration_ms|             energy|                  id|   instrumentalness|              key|           liveness|            loudness|                  name|       popularity|        speechiness|             tempo|      release_year|
+-------+-------------------+-------------------+--------------------+-------------------+------------------+-------------------+--------------------+-------------------+-----------------+-------------------+--------------------+----------------------+-----------------+-------------------+------------------+------------------+
|  count|    

save data to a local folder

In [45]:
clean_df.write \
    .mode("overwrite")\
    .parquet("data/musicInfo")